## Objective- join tables togeather with query and clean for EDA 

#Steps - 
1 - Notebook Querys joined data from BQ tables 

2 - nulls and duplicates are taken care off 
3- save cleaned data to drive 

# Next Steps - 
- add shooting data and EDA 


In [2]:
from google.cloud import storage

In [3]:
import google.cloud

In [38]:
import os
import pandas as pd
import pandas_gbq
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery
from config import creds

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= creds
# "/Users/robertrustia/Desktop/soccer/mls_info/g_k/san-jose-earthquakes-dev-356d71186392.json"


credentials = service_account.Credentials.from_service_account_file(creds)
# client = language.LanguageServiceClient(credentials=credentials)


client = storage.Client()
project_id = 'san-jose-earthquakes-dev'

client = bigquery.Client(project_id)

In [21]:
from google.cloud import auth
auth.authenticate_user()
print('Authenticated.')


ImportError: cannot import name 'auth' from 'google.cloud' (unknown location)

In [22]:
# import pandas as pd





# # from google.colab import auth
# # auth.authenticate_user()
# # print('Authenticated.')

# project_id = 'san-jose-earthquakes-dev'

# client = bigquery.Client(project_id)

# query to get all merged features from fb_ref tables 

In [39]:
sql = """
WITH
  -- opposing team stats
  opp AS (
  SELECT
    Squad AS Squad_from_op,
    Opposing_Poss,
    Opposing_Touches
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.opposing_possesion` ),
  -- Mich with aerial stats
  mish AS (
  SELECT
    Player as mish_player,
    Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    Interceptions as def_interceptions,
    Tackles_won as from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won,
     Aerial_Duels_lost,
    Aerial_win_pct 
    from 
    `san-jose-earthquakes-dev.MLS_2021_fbref.Miscellaneous_stats` ),
  -- defensive stats
  def AS (
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.Defensive_stats` ),
  -- player passing stats
  pass AS (
  SELECT
    Player,
    fb_id,
    player_name,
    Nineties,
    CAST(xA AS float64) AS xA,
    CAST(Key_Pass AS int64) AS Key_pass,
    CAST(Comp_prog AS float64) AS Comp_prog,
    Total_Cmp,
    Total_Att,
    Total_Cmp_pct,
    Total_Tot_Dist,
    Total_Prg_Dist,
    Short_Cmp,
    Short_Att,
    Short_Cmp_pct,
    Medium_Cmp,
    Medium_Att,
    Medium_Cmp_pct,
    Long_Cmp,
    Long_Att,
    Long_Cmp_pct,
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.passing_stats` ),
  --player possesion stats
  posses AS(
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.possesion_stats` )
  --
  ### main query below
  --
SELECT
  pass.fb_id,
  pass.player_name,
  opp.Squad_from_op,
  opp.Opposing_Poss,
  opp.Opposing_Touches,
  -- def stats
  def.Nation,
  def.Position,
  pass.Nineties,
  pass.xA,
  pass.Key_pass,
  pass.Comp_prog,
  pass.Total_Cmp,
  pass.Total_Att,
  pass.Total_Cmp_pct,
  pass.Total_Tot_Dist,
  pass.Total_Prg_Dist,
  pass.Short_Cmp,
  pass.Short_Att,
  pass.Short_Cmp_pct,
  pass.Medium_Cmp,
  pass.Medium_Att,
  pass.Medium_Cmp_pct,
  pass.Long_Cmp,
  pass.Long_Att,
  pass.Long_Cmp_pct,
  -- posses below
  posses.touches,
  posses.touches_def_box,
  posses.touches_def_3rd,
  posses.touches_mid_3rd,
  posses.touches_att_3rd,
  posses.touches_att_box,
  posses.touch_live,
  posses.dribble_Succ,
  posses.dribble_Att,
  posses.dribble_Succ_pct,
  posses.num_dribble_past,
  posses.dribble_megs,
  posses.carries,
  posses.carries_dist,
  posses.carries_prog_dist,
  posses.carries_prog,
  posses.carries_att_3rd,
  posses.carries_att_box,
  posses.miss_control,
  posses.dispossed,
  posses.recep_targ,
  posses.recep_succ,
  posses.recept_pct,
  posses.prog_pass_rec

  -- mish

  Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    def_interceptions,
    from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won Aerial_Duels_lost,
    Aerial_win_pct 
FROM
  def
LEFT JOIN
  pass
ON
  def.Player = pass.Player
left join 
  mish 
on 
  def.Player = mish.mish_player
LEFT JOIN
  posses
ON
  def.Player = posses.Player
LEFT JOIN
  opp
ON
  def.Team = opp.Squad_from_op
ORDER BY
  Key_pass DESC

"""
df = pandas_gbq.read_gbq(sql,project_id=project_id)

In [41]:
# updated with blocks 
sql = """
WITH
  -- opposing team stats
  opp AS (
  SELECT
    Squad AS Squad_from_op,
    Opposing_Poss,
    Opposing_Touches
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.opposing_possesion` ),
  -- Mich with aerial stats
  mish AS (
  SELECT
    Player as mish_player,
    Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    Interceptions as def_interceptions,
    Tackles_won as from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won,
     Aerial_Duels_lost,
    Aerial_win_pct 
    from 
    `san-jose-earthquakes-dev.MLS_2021_fbref.Miscellaneous_stats` ),
  -- defensive stats
  def AS (
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.Defensive_stats` ),
  -- player passing stats
  pass AS (
  SELECT
    Player,
    fb_id,
    player_name,
    Nineties,
    CAST(xA AS float64) AS xA,
    CAST(Key_Pass AS int64) AS Key_pass,
    CAST(Comp_prog AS float64) AS Comp_prog,
    Total_Cmp,
    Total_Att,
    Total_Cmp_pct,
    Total_Tot_Dist,
    Total_Prg_Dist,
    Short_Cmp,
    Short_Att,
    Short_Cmp_pct,
    Medium_Cmp,
    Medium_Att,
    Medium_Cmp_pct,
    Long_Cmp,
    Long_Att,
    Long_Cmp_pct,
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.passing_stats` ),
  --player possesion stats
  posses AS(
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.possesion_stats` )
  --
  ### main query below
  --
SELECT
  pass.fb_id,
  pass.player_name,
  opp.Squad_from_op,
  opp.Opposing_Poss,
  opp.Opposing_Touches,
  -- def stats
  
  def.Player,
  def.Nation,
  def.Position,

  def.Total_tackles,
  def.Tackles_Won,	
  def.Tackles_Def_3rd,	
  def.Tackles_Mid_3rd,	
  def.Tackles_Att_3rd,	
  def.Num_Dribblers_tackled,	
  def.Num_Dribbled_past,	
  def.total_pressing,	
  def.Won_ball_5_sec,	
  def.pct_won,	
  def.pres_Def_3rd,	
  def.pres_Mid_3rd,	
  def.pres_Att_3rd,	
  def.Blocks,	
  def.Blocked_shots,	
  def.Blocked_SOT,	
  def.Blocked_pass,	
  def.Interceptions,	
  def.Tackles_and_Ints,	
  def.Clearences,
  pass.Nineties,
  pass.xA,
  pass.Key_pass,
  pass.Comp_prog,
  pass.Total_Cmp,
  pass.Total_Att,
  pass.Total_Cmp_pct,
  pass.Total_Tot_Dist,
  pass.Total_Prg_Dist,
  pass.Short_Cmp,
  pass.Short_Att,
  pass.Short_Cmp_pct,
  pass.Medium_Cmp,
  pass.Medium_Att,
  pass.Medium_Cmp_pct,
  pass.Long_Cmp,
  pass.Long_Att,
  pass.Long_Cmp_pct,
  -- posses below
  posses.touches,
  posses.touches_def_box,
  posses.touches_def_3rd,
  posses.touches_mid_3rd,
  posses.touches_att_3rd,
  posses.touches_att_box,
  posses.touch_live,
  posses.dribble_Succ,
  posses.dribble_Att,
  posses.dribble_Succ_pct,
  posses.num_dribble_past,
  posses.dribble_megs,
  posses.carries,
  posses.carries_dist,
  posses.carries_prog_dist,
  posses.carries_prog,
  posses.carries_att_3rd,
  posses.carries_att_box,
  posses.miss_control,
  posses.dispossed,
  posses.recep_targ,
  posses.recep_succ,
  posses.recept_pct,
  posses.prog_pass_rec

  -- mish

  Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    def_interceptions,
    from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won Aerial_Duels_lost,
    Aerial_win_pct 
FROM
  def
LEFT JOIN
  pass
ON
  def.Player = pass.Player
left join 
  mish 
on 
  def.Player = mish.mish_player
LEFT JOIN
  posses
ON
  def.Player = posses.Player
LEFT JOIN
  opp
ON
  def.Team = opp.Squad_from_op
ORDER BY
  Key_pass DESC

"""
df = pandas_gbq.read_gbq(sql,project_id=project_id)



/Users/robertrustia/opt/anaconda3/envs/devenv/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1983: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  create_bqstorage_client=create_bqstorage_client,


In [42]:
info= pd.DataFrame({"values":{col:df[col].unique() for col in df},
              'type':{col:df[col].dtype for col in df},
              'unique values':{col:len(df[col].unique()) for col in df},
              'NA values':{col:str(round(sum(df[col].isna())/len(df),2))+'%' for col in df},
              'Duplicated Values':{col:sum(df[col].duplicated()) for col in df}
             })

info.T

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Player,Nation,Position,Total_tackles,Tackles_Won,...,Offsides,Crosses,def_interceptions,from_mish_tackles_won,PK_won,PK_conceded,OG,Recoveries,Aerial_Duels_lost,Aerial_win_pct
values,"[11c30a46, acd47bc0, 614b26e3, 0adeafaa, f99f3...","[Carles Gil, Julian Gressel, Emanuel Reynoso, ...","[New England, D.C. United, Minnesota Utd, Real...","[49.1, 54.0, 49.9, 46.5, 51.0, 48.3, 47.1, 42....","[20024, 20526, 20887, 19279, 19332, 20501, 193...","[Carles Gil\11c30a46, Julian Gressel\acd47bc0,...","[es ESP, de GER, ar ARG, sk SVK, am ARM, us US...","[MF, DF, MFFW, FWMF, FW, FWDF, DFMF, MFDF, DFF...","[10, 46, 59, 24, 48, 56, 32, 35, 47, 54, 90, 2...","[4, 19, 43, 16, 26, 42, 20, 27, 39, 65, 24, 21...",...,"[0, 4, 2, 1, 5, 8, 3, 14, 26, 10, 7, 6, 22, 15...","[114, 144, 69, 46, 70, 59, 100, 62, 126, 41, 5...","[23, 68, 27, 25, 28, 30, 44, 53, 40, 21, 18, 3...","[4, 19, 43, 16, 26, 42, 20, 27, 39, 65, 24, 21...","[0, 1, 3, 2]","[0, 1, 2, 3]","[0, 1, 2]","[164, 251, 198, 203, 167, 178, 153, 163, 243, ...","[4, 30, 18, 10, 5, 6, 22, 26, 19, 8, 17, 15, 1...","[17.4, 44.8, 28.6, 12.9, 23.3, 27.8, 13.6, 44...."
type,object,object,object,float64,Int64,object,object,object,object,object,...,object,object,object,object,object,object,object,object,object,object
unique values,730,730,27,24,27,730,85,10,88,63,...,23,82,80,63,4,4,3,241,86,285
NA values,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,...,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.11%
Duplicated Values,195,195,898,901,898,195,840,915,837,862,...,902,843,845,862,921,921,922,684,839,640


In [43]:
df.describe()

,Opposing_Poss,Opposing_Touches,xA,Key_pass,Comp_prog
count,925.000000,925.000000,925.000000,925.000000,925.000000
mean,50.083027,20247.580541,1.079351,10.406486,37.052973
std,3.152554,1037.474638,1.569396,14.663216,41.359008
min,42.600000,17449.000000,0.000000,0.000000,0.000000
25%,48.300000,19650.000000,0.100000,1.000000,5.000000
50%,50.200000,20480.000000,0.400000,5.000000,21.000000
75%,51.900000,21096.000000,1.500000,14.000000,55.000000
max,55.100000,22416.000000,12.200000,134.000000,277.000000


In [44]:
df.fb_id.nunique()

730

In [45]:
df.duplicated(subset=['fb_id']).sum()

195

In [46]:
df.duplicated(subset=['fb_id']).value_counts()

False    730
True     195
dtype: int64

In [47]:
dups = df[df.duplicated(subset=['fb_id'])]

In [48]:
df.iloc[921:925]

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Player,Nation,Position,Total_tackles,Tackles_Won,...,Offsides,Crosses,def_interceptions,from_mish_tackles_won,PK_won,PK_conceded,OG,Recoveries,Aerial_Duels_lost,Aerial_win_pct
921,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,Florian Valot\ad454d78,fr FRA,MFFW,6,3,...,0,3,2,1,0,0,0,9,2,66.7
922,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,Florian Valot\ad454d78,fr FRA,MFFW,6,3,...,0,3,2,1,0,0,0,9,2,66.7
923,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,Florian Valot\ad454d78,fr FRA,MFFW,6,3,...,0,3,9,3,0,0,0,25,6,75
924,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,Florian Valot\ad454d78,fr FRA,MFFW,6,3,...,0,3,9,3,0,0,0,25,6,75


In [49]:
dups['player_name'].value_counts()

Mark-Anthony Kaye    15
Nicolas Benezet      15
Brian White          15
Erik Hurtado         15
Ramón Ábila          15
Jeremy Ebobisse      15
Florian Valot        15
Corey Baird          15
Amar Sejdic          15
Florian Jungwirth    15
Sebastien Ibeagha    15
Dominique Badji      15
Liam Fraser          15
Name: player_name, dtype: int64

In [50]:
dups['player_name'].unique()

array(['Mark-Anthony Kaye', 'Nicolas Benezet', 'Brian White',
       'Erik Hurtado', 'Ramón Ábila', 'Jeremy Ebobisse', 'Florian Valot',
       'Corey Baird', 'Amar Sejdic', 'Florian Jungwirth',
       'Sebastien Ibeagha', 'Dominique Badji', 'Liam Fraser'],
      dtype=object)

In [51]:
# Considering certain columns for dropping duplicates
df.drop_duplicates(subset=['player_name'],keep='first',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 920
Data columns (total 84 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fb_id                  730 non-null    object 
 1   player_name            730 non-null    object 
 2   Squad_from_op          730 non-null    object 
 3   Opposing_Poss          730 non-null    float64
 4   Opposing_Touches       730 non-null    Int64  
 5   Player                 730 non-null    object 
 6   Nation                 730 non-null    object 
 7   Position               730 non-null    object 
 8   Total_tackles          730 non-null    object 
 9   Tackles_Won            730 non-null    object 
 10  Tackles_Def_3rd        730 non-null    object 
 11  Tackles_Mid_3rd        730 non-null    object 
 12  Tackles_Att_3rd        730 non-null    object 
 13  Num_Dribblers_tackled  730 non-null    object 
 14  Num_Dribbled_past      730 non-null    object 
 15  total_

In [52]:
df.duplicated(subset=['fb_id']).value_counts()

False    730
dtype: int64

look for null values in dataframe

In [53]:



missing = [df.isnull().sum(),100*df.isnull().mean()]

m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

m = pd.DataFrame(missing)
m = m.T
m.columns=['count','%']
m =m.sort_values(by='count',ascending=False)
m.head(10)

,count,%
dribble_Succ_pct,106.0,14.520548
Aerial_win_pct,90.0,12.328767
Long_Cmp_pct,24.0,3.287671
pct_won,16.0,2.191781
Short_Cmp_pct,5.0,0.684932
Medium_Cmp_pct,5.0,0.684932
recept_pct,2.0,0.273973
Total_Cmp_pct,2.0,0.273973
Nation,0.0,0.000000
miss_control,0.0,0.000000


look to drop players with nulls in features what are less than 1% missing 

look at a couple dribble succ_pct players and figure out why null and if mean is decent fill 

In [54]:
df.dropna(subset=['Long_Cmp_pct','Short_Cmp_pct','Medium_Cmp_pct','Total_Cmp_pct','recept_pct'],inplace=True )
missing = [df.isnull().sum(),100*df.isnull().mean()]

m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

m = pd.DataFrame(missing)
m = m.T
m.columns=['count','%']
m =m.sort_values(by='count',ascending=False)
m.head(10)

,count,%
dribble_Succ_pct,91.0,12.907801
Aerial_win_pct,80.0,11.347518
pct_won,15.0,2.127660
dribble_Att,0.0,0.000000
carries_prog,0.0,0.000000
carries_prog_dist,0.0,0.000000
carries_dist,0.0,0.000000
carries,0.0,0.000000
dribble_megs,0.0,0.000000
num_dribble_past,0.0,0.000000


In [55]:
df[['dribble_Succ_pct','Aerial_win_pct']].describe()

,dribble_Succ_pct,Aerial_win_pct
count,614,625
unique,197,283
top,100,50
freq,61,28


In [56]:
df.Position.value_counts()

DF      227
MF      115
FWMF     96
FW       87
GK       65
MFFW     64
DFMF     18
MFDF     14
FWDF     10
DFFW      9
Name: Position, dtype: int64

In [58]:
# state_price_means = ski_data.groupby('state')[['AdultWeekday', 'AdultWeekend']].mean()
# state_price_means.head()

# df.groupby('dummy')['returns'].agg(Mean='mean', Sum='sum')
import numpy as np

fills =df[['Position','dribble_Succ_pct','Aerial_win_pct']]
fills[['dribble_Succ_pct','Aerial_win_pct']] = fills[['dribble_Succ_pct','Aerial_win_pct']].astype(float)
fills.info()
fills= fills.groupby('Position').agg({'dribble_Succ_pct': [np.mean, np.sum],'Aerial_win_pct': [np.mean, np.sum]})
fills.T.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 919
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Position          705 non-null    object 
 1   dribble_Succ_pct  614 non-null    float64
 2   Aerial_win_pct    625 non-null    float64
dtypes: float64(2), object(1)
memory usage: 22.0+ KB


Position                         DF        DFFW         DFMF           FW  \
dribble_Succ_pct mean     65.003431   57.100000    69.005882    49.830588   
                 sum   13260.700000  513.900000  1173.100000  4235.600000   
Aerial_win_pct   mean     56.550909   40.688889    47.868750    36.162791   
                 sum   12441.200000  366.200000   765.900000  3110.000000   

Position                     FWDF         FWMF           GK           MF  \
dribble_Succ_pct mean   56.966667    54.151042    90.909091    64.974074   
                 sum   512.700000  5198.500000  1000.000000  7017.200000   
Aerial_win_pct   mean   42.355556    30.815789    75.000000    46.703636   
                 sum   381.200000  2927.500000   300.000000  5137.400000   

Position                     MFDF         MFFW  
dribble_Succ_pct mean   64.015385    56.140323  
                 sum   832.200000  3480.700000  
Aerial_win_pct   mean   50.850000    35.640323  
                 sum   711.900000  2209.700000

In [ ]:
#EDA

## look at one example 
# df[df['Squad_from_op']=='San Jose']


In [59]:
df[['dribble_Succ_pct','Aerial_win_pct']] = df[['dribble_Succ_pct','Aerial_win_pct']].astype(float)

df['dribble_Succ_pct'].fillna((df['dribble_Succ_pct'].mean()), inplace=True)
df['Aerial_win_pct'].fillna((df['Aerial_win_pct'].mean()), inplace=True)

In [62]:
df.to_csv('../data/with_def_p_stats.csv')

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 919
Data columns (total 84 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fb_id                  705 non-null    object 
 1   player_name            705 non-null    object 
 2   Squad_from_op          705 non-null    object 
 3   Opposing_Poss          705 non-null    float64
 4   Opposing_Touches       705 non-null    Int64  
 5   Player                 705 non-null    object 
 6   Nation                 705 non-null    object 
 7   Position               705 non-null    object 
 8   Total_tackles          705 non-null    object 
 9   Tackles_Won            705 non-null    object 
 10  Tackles_Def_3rd        705 non-null    object 
 11  Tackles_Mid_3rd        705 non-null    object 
 12  Tackles_Att_3rd        705 non-null    object 
 13  Num_Dribblers_tackled  705 non-null    object 
 14  Num_Dribbled_past      705 non-null    object 
 15  total_

use bar graphs to observe distriubtion of key features 